In [102]:
# """
# The current code given is for the Assignment 1.
# You will be expected to use this to make trees for:
# > discrete input, discrete output
# > real input, real output
# > real input, discrete output
# > discrete input, real output
# """
# from dataclasses import dataclass
# from typing import Literal
# import sys
# sys.path.append("..")
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from tree.utils import *

# N = 30 # no: of samples
# P = 5 # no : of features
# X = pd.DataFrame({i: pd.Series(np.random.randint(P, size=N), dtype="category") for i in range(5)})
# y = pd.Series(np.random.randn(N))

# np.random.seed(42)

# class Node:
#     def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
#         self.feature = feature
#         self.threshold = threshold
#         self.left = left
#         self.right = right
#         self.value = value

# @dataclass
# class DecisionTree:
#     criterion: Literal["information_gain", "gini_index"]  # criterion won't be used for regression
#     max_depth: int  # The maximum depth the tree can grow to

#     def __init__(self, criterion, max_depth=5):
#         self.criterion = criterion
#         self.max_depth = max_depth
        
#     def fit(self, X: pd.DataFrame, y: pd.Series) -> None:
#         """
#         Function to train and construct the decision tree
#         """
        
#         # If you wish your code can have cases for different types of input and output data (discrete, real)
#         # Use the functions from utils.py to find the optimal attribute to split upon and then construct the tree accordingly.
#         # You may(according to your implemetation) need to call functions recursively to construct the tree. 

#         pass

#     def predict(self, X: pd.DataFrame) -> pd.Series:
#         """
#         Funtion to run the decision tree on test inputs
#         """

#         # Traverse the tree you constructed to return the predicted values for the given test inputs.

#         pass

#     def plot(self) -> None:
#         """
#         Function to plot the tree

#         Output Example:
#         ?(X1 > 4)
#             Y: ?(X2 > 7)
#                 Y: Class A
#                 N: Class B
#             N: Class C
#         Where Y => Yes and N => No
#         """
#         pass

In [104]:
from dataclasses import dataclass
from typing import Literal
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tree.utils import *

def MSE(series):
    series_MSE = 0
    mean = np.mean(series)
    for i in series:
        series_MSE += (mean - i)**2
    return series_MSE
    
def information_gain(Y: pd.Series, attr: pd.Series) -> float:
    """
    Function to calculate the information gain
    """
    # if check_ifreal(Y) and not check_ifreal(attr):
    dist_target = attr.value_counts(normalize=True)
    normalized_target = dist_target/dist_target.sum()
    IG = MSE(Y)
    for value, p in normalized_target.items():
        series = Y[attr == value]
        IG = IG - p * MSE(series) # 1e-6 is added to avoid log(0)
    return IG
    
def opt_split_attribute(X: pd.DataFrame, y: pd.Series):
    IG_list = {}
    for i in X:
        IG_list[i] = information_gain(y, X[i])
    IGvalues = list(IG_list.values())
    IGkeys = list(IG_list.keys())
    return IGkeys[IGvalues.index(max(IGvalues))]
    
N = 30 # no: of samples
P = 5 # no : of features
X = pd.DataFrame({i: pd.Series(np.random.randint(P, size=N), dtype="category") for i in range(5)})
y = pd.Series(np.random.randn(N))

In [105]:
print(opt_split_attribute(X,y))

0


In [106]:
def all_elements_equal(series):
    return series.nunique() == 1

class Node:
    def __init__(self, X: pd.DataFrame, y: pd.Series, max_depth = 5, name=None):
        self.name = name
        feature = opt_split_attribute(X,y)
        self.feature = feature
        self.value = None
        self.child = []
        if max_depth == 1 or all_elements_equal(y):
            avg = 0
            for i in y:
                avg += i
            avg = avg/len(y)
            self.value = avg
            return None
        for each in X[feature].unique():
            X_new = X[X[feature] == each]
            y_new = y[X[feature] == each]
            X_new = X_new.drop(feature, axis=1)
            self.child.append(Node(X_new,y_new,max_depth-1,each))

@dataclass
class DecisionTree:
    criterion: Literal["information_gain", "gini_index"]  # criterion won't be used for regression
    max_depth: int  # The maximum depth the tree can grow to

    def __init__(self, criterion, max_depth=5):
        self.criterion = criterion
        self.max_depth = max_depth
        
    def fit(self, X: pd.DataFrame, y: pd.Series) -> None:
        """
        Function to train and construct the decision tree
        """
        self.tree = Node(X,y,self.max_depth,"root")
        # If you wish your code can have cases for different types of input and output data (discrete, real)
        # Use the functions from utils.py to find the optimal attribute to split upon and then construct the tree accordingly.
        # You may(according to your implemetation) need to call functions recursively to construct the tree. 
        pass
    def predict(self, X: pd.DataFrame) -> pd.Series:
        """
        Function to run the decision tree on test inputs
        """
        y_hat = []
        for _, row in X.iterrows():
            node = self.tree
            while node.value is None:
                feature_value = row[node.feature]
                for child in node.child:
                    if child.name == feature_value:
                        node = child
                        break
            y_hat.append(node.value)
        return pd.Series(y_hat)
        
    def plot(self) -> None:
        """
        Function to plot the tree

        Output Example:
        ?(X1 > 4)
            Y: ?(X2 > 7)
                Y: Class A
                N: Class B
            N: Class C
        Where Y => Yes and N => No
        """
        pass


In [107]:
for criteria in ["information_gain"]:
    tree = DecisionTree(criterion=criteria)  # Split based on Inf. Gain
    tree.fit(X, y)
    y_hat = tree.predict(X)
    tree.plot()
    print("Criteria :", criteria)
    # print("RMSE: ", rmse(y_hat, y))
    # print("MAE: ", mae(y_hat, y))

Criteria : information_gain
